<a href="https://colab.research.google.com/github/jyun0618/EPOCH_FinalProject/blob/main/modeling/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/X_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/y_train.csv')
X_val = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/X_val.csv')
y_val = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/y_val.csv')
X_test = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/X_test.csv')

sample_submission = pd.read_csv('/content/drive/MyDrive/EPOCH/Final Project/data/sample_submission.csv')

## 1. 모델 학습

## 2. 하이퍼파라미터 튜닝

### 2-1. Random Forest Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import joblib  # 모델 저장용 라이브러리

# 모델 초기화
rf = RandomForestClassifier()

# 하이퍼파라미터 범위 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_log_loss')

# GridSearch 수행
grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 테스트 데이터로 성능 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict_proba(X_val)
print("log loss score:", log_loss(y_val, y_pred))



# 모델 저장
model_filename = 'best_randomforest_model.pkl'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

# 모델 로드
loaded_model = joblib.load('best_randomforest_model.pkl')
print("Model loaded successfully.")

# 로드한 모델로 테스트 데이터 예측
loaded_y_pred_rf = loaded_model.predict_proba(X_test)

### 2-2. XGBoost Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# XGBoost 모델 초기화
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_log_loss',
    cv=5,  # 5-fold 교차 검증
    verbose=1,
    n_jobs=-1
)

# GridSearch 수행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 테스트 데이터 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict_proba(X_val)
logloss = log_loss(y_val, y_pred)
print("log loss score:", logloss)



# 모델 저장
model_filename = 'best_xgboost_model.pkl'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

# 모델 로드
loaded_model = joblib.load(model_filename)
print("Model loaded successfully.")

# 로드한 모델로 테스트 데이터 재평가
loaded_y_pred_xg = loaded_model.predict_proba(X_test)

array([[0.01118921, 0.34227154, 0.6465393 ],
       [0.04113001, 0.06600755, 0.89286244],
       [0.02133671, 0.24997938, 0.72868395],
       ...,
       [0.02725354, 0.08757222, 0.8851743 ],
       [0.02785655, 0.16385484, 0.80828863],
       [0.0230907 , 0.28631285, 0.69059646]], dtype=float32)

In [ ]:
# 제출 파일 저장
sample_submission.iloc[:,1:] = loaded_y_pred_xg
sample_submission.to_csv('best_xgboost_model.csv', index=False)

<ipython-input-21-44ad6ecc5737>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.01118921 0.04113001 0.02133671 ... 0.02725354 0.02785655 0.0230907 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sample_submission.iloc[:,1:] = loaded_y_pred_xg
<ipython-input-21-44ad6ecc5737>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.34227154 0.06600755 0.24997938 ... 0.08757222 0.16385484 0.28631285]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sample_submission.iloc[:,1:] = loaded_y_pred_xg
<ipython-input-21-44ad6ecc5737>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.6465393  0.89286244 0.72868395 ... 0.8851743  0.80828863 0.69059646]' has dtype incompatible with int64, please explic